# Домашнее задание 5

Работа с базой `book_simple.db`. 

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=160L6VoP2kz8mJfpFuLWF6_SdoVcvkT-J' width="400" height="400" />
<figcaption>Структура базы book </figcaption></center>
</figure>

Ознакомиться со структурой данных можно с ипользованием, например, программы 
DBeaмer. 

Для работы с базой допускается использовать только возможности SQL. 
Библиотеку Pandas допускается использовать только просмотра результата.

Шаг 1.  Изучите материалы лекционных и практических занятий по теме 6.1-6.4.

Шаг 2. Получите результирующие множества из БД в соответствии с следующими условиями выборки, сортировки, группировки данных:

1. Выведите авторов, имеющихся в базе?
2. Найдите всех авторов с инициалами "С. А.". Автор в базе определяется так: И. О. Фамилия
3. Найдите все издательства, название которых состоит из двух и более слов.
4. Выведите список книг, выпущенных в издательстве "Мир".
5. Выведите количество книг, принадлежащих каждой из категорий. Результат отсортировать по возрастанию.
6. Выведите количество книг, выпущенных в каждом издательстве. В итоговом списке оставьте только те издательства, которые выпустили 3 и более книг. Результат отсортировать по убыванию.
7. Выведите список книг, автором которых являлся <<С. А. Айвазян>>.
8. В каких издательствах и сколько выпускались книги за авторством <<С. А. Айвазян>>. Результат отсортировать по убыванию.

Шаг 3. Результат каждого запроса запишите в виде таблицы в БД book.db.

Шаг 4. Опубликуйте итоговый файл БД расширения sql, прикрепив его в этом задании.

In [3]:
# Подключение библиотек
import sqlite3
import pandas as pd

# Подключение к БД
con_simple = sqlite3.connect("book_simple.db")
con_result = sqlite3.connect("book.db")

# Задание курсора БД
cur_simple = con_simple.cursor()
cur_result = con_result.cursor()

# Получить список всех таблиц в базе данных
querty_show_table = """
SELECT name 
FROM sqlite_master
WHERE type = 'table'
ORDER BY name;
"""

info_db = cur_simple.execute(querty_show_table)
info_db.fetchall()

[('author',), ('authorbook',), ('book',), ('category',), ('publisher',)]

##### 1. Выведите авторов, имеющихся в базе.

In [4]:
querty_author = """
SELECT DISTINCT name AS author
  FROM author;
"""

name_authors = pd.read_sql(querty_author, con_simple)

# Записать результат запроса в новую таблицу БД
name_authors.to_sql("name_authors", con=con_result, index=False)
name_authors

,author
0,М. А. Айзерман
1,Ф. Т. Алескеров
2,Г. В. Савицкая
3,В. Кельтон
4,А. Лоу
...,...
1005,В. Н. Касьянов
1006,Ф. Коуэлл
1007,В. В. Ступкин
1008,М. А. Тынкевич


##### 2. Найдите всех авторов с инициалами "С. А.". Автор в базе определяется так: И. О. Фамилия.

In [5]:
querty_initials = """
SELECT DISTINCT name AS initials_s_a
  FROM author
WHERE name LIKE ("С. А. %");
"""

initials = pd.read_sql(querty_initials, con_simple)
initials.to_sql("initials", con=con_result, index=False)
initials

,initials_s_a
0,С. А. Айвазян
1,С. А. Орловского
2,С. А. Смоляк
3,С. А. Айвазян
4,С. А. Калмыков
5,С. А. Дубровский
6,С. А. Аристов
7,С. А. Иванков
8,С. А. Айвазяна
9,С. А. Горбатков


##### 3. Найдите все издательства, название которых состоит из двух и более слов.

In [6]:
querty_publishing = """
SELECT name AS publisher
  FROM publisher
WHERE name LIKE ("_% _%");
"""

publishing = pd.read_sql(querty_publishing, con_simple)
publishing.to_sql("publishing", con=con_result, index=False)
publishing

,publisher
0,Новое знание
1,Экономическая школа
2,Высш. шк.
3,Радио и связь
4,Гyманитарный Центр
...,...
58,ИПП Куна
59,ДМК Пресс
60,Новое звание
61,ИПУ РАН


##### 4. Выведите список книг, выпущенных в издательстве "Мир".

In [7]:
querty_mir = """
SELECT book.title AS book, publisher.name AS publisher
  FROM book, publisher
WHERE publisher.name LIKE ("Мир");
"""

mir = pd.read_sql(querty_mir, con_simple)
mir.to_sql("mir", con=con_result, index=False)
mir

,book,publisher
0,Выбор вариантов: основы теории,Мир
1,Экономический анализ,Мир
2,Имитационное моделирование. Классика CS,Мир
3,Математическая экономия,Мир
4,Управление производством при нечеткой исходной...,Мир
...,...,...
823,Теория графов: алгоритмы обработки деревьев,Мир
824,Микроэкономика. Принципы и анализ,Мир
825,Проектирование интегрированных систем библиоте...,Мир
826,Экономико-математические методы,Мир


##### 5. Выведите количество книг, принадлежащих каждой из категорий. Результат отсортировать по возрастанию.

In [8]:
querty_count_category = """
SELECT category.title AS category, COUNT(*) AS amount
  FROM category
INNER JOIN book ON category.id = book.category_id
GROUP BY category.title
ORDER BY amount;
"""

count_category = pd.read_sql(querty_count_category, con_simple)
count_category.to_sql("count_category", con=con_result, index=False)
count_category

,category,amount
0,Экономические науки,198
1,Информационные технологии,201
2,Теория вероятностей и математическая статистика,214
3,Математическая экономия,215


##### 6. Выведите количество книг, выпущенных в каждом издательстве. В итоговом списке оставьте только те издательства, которые выпустили 3 и более книг. Результат отсортировать по убыванию.

In [9]:
querty_count_book = """
SELECT publisher.name AS publisher, COUNT(*) AS amount
  FROM publisher
INNER JOIN book ON publisher.id = book.publisher_id
GROUP BY publisher.name
HAVING COUNT(*) >= 3
ORDER BY amount DESC;
"""

count_book = pd.read_sql(querty_count_book, con_simple)
count_book.to_sql("count_book", con=con_result, index=False)
count_book

,publisher,amount
0,Наука,152
1,Мир,84
2,Финансы и статистика,38
3,ФИЗМАТЛИТ,28
4,Радио и связь,26
5,Статистика,26
6,Питер,23
7,Вильямс,20
8,Высш. шк.,19
9,Машиностроение,15


##### 7. Выведите список книг, автором которых являлся <<С. А. Айвазян>>.

In [10]:
querty_ayvazyan = """
SELECT book.title AS book, author.name AS author
  FROM author
INNER JOIN authorbook ON author.id = authorbook.author_id
INNER JOIN book ON authorbook.book_id = book.id
WHERE author.name LIKE ("С. А. Айвазян");
"""

ayvazyan = pd.read_sql(querty_ayvazyan, con_simple)
ayvazyan.to_sql("ayvazyan", con=con_result, index=False)
ayvazyan

,book,author
0,Социальные индикаторы,С. А. Айвазян
1,Прикладная статистика. Основы эконометрики: Уч...,С. А. Айвазян
2,Прикладная статистика: Классификации и снижени...,С. А. Айвазян
3,Прикладная статистика: Исследование зависимостей,С. А. Айвазян
4,Прикладная статистика: Основы моделирования и ...,С. А. Айвазян
5,Статистика. Вероятность. Экономика,С. А. Айвазян
6,Классификация многомерных наблюдений,С. А. Айвазян
7,Прикладная статистика в задачах и упражнениях,С. А. Айвазян


##### 8. В каких издательствах и сколько выпускались книги за авторством <<С. А. Айвазян>>. Результат отсортировать по убыванию.

In [11]:
querty_count_ayvazyan = """
SELECT author.name AS author, publisher.name AS publisher, COUNT(*) AS amount
  FROM author
INNER JOIN authorbook ON author.id = authorbook.author_id
INNER JOIN book ON authorbook.book_id = book.id
INNER JOIN publisher ON book.publisher_id = publisher.id
WHERE author.name LIKE ("С. А. Айвазян")
GROUP BY publisher.name
ORDER BY amount DESC;
"""

count_ayvazyan = pd.read_sql(querty_count_ayvazyan, con_simple)
count_ayvazyan.to_sql("count_ayvazyan", con=con_result, index=False)
count_ayvazyan

,author,publisher,amount
0,С. А. Айвазян,Финансы и статистика,3
1,С. А. Айвазян,ЮНИТИ-ДАНА,2
2,С. А. Айвазян,Наука,1
3,С. А. Айвазян,Статистика,1
4,С. А. Айвазян,Юнити-Дана,1


In [12]:
# Получить список всех таблиц в новой базе данных
cur_result.execute(querty_show_table).fetchall()

[('ayvazyan',),
 ('count_ayvazyan',),
 ('count_book',),
 ('count_category',),
 ('initials',),
 ('mir',),
 ('name_authors',),
 ('publishing',)]

In [13]:
# Закрытие БД
cur_simple.close()
cur_result.close()